# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 22 2022 10:59AM,247050,10,MSP214193,"Methapharm, Inc.",Released
1,Sep 22 2022 10:58AM,247049,10,MSP213929,"Methapharm, Inc.",Released
2,Sep 22 2022 10:56AM,247047,20,ALUR023867294,Alumier Labs Inc.,Released
3,Sep 22 2022 10:56AM,247047,20,ALUR074187133,Alumier Labs Inc.,Released
4,Sep 22 2022 10:56AM,247047,20,ALUR340006790,Alumier Labs Inc.,Released
5,Sep 22 2022 10:56AM,247047,20,ALUR462930243,Alumier Labs Inc.,Released
6,Sep 22 2022 10:56AM,247047,20,ALUR941481874,Alumier Labs Inc.,Released
7,Sep 22 2022 10:56AM,247046,10,MSP214213,"Methapharm, Inc.",Released
8,Sep 22 2022 10:56AM,247046,10,MSP214224,"Methapharm, Inc.",Released
9,Sep 22 2022 10:56AM,247046,10,MSP214229,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
29,247044,Released,1
30,247046,Released,8
31,247047,Released,5
32,247049,Released,1
33,247050,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247044,NaN,NaN,1.0
247046,NaN,NaN,8.0
247047,NaN,NaN,5.0
247049,NaN,NaN,1.0
247050,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246888,0.0,1.0,0.0
246889,0.0,1.0,0.0
246890,0.0,0.0,1.0
246933,0.0,1.0,0.0
246951,5.0,1.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246888,0,1,0
246889,0,1,0
246890,0,0,1
246933,0,1,0
246951,5,1,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246888,0,1,0
1,246889,0,1,0
2,246890,0,0,1
3,246933,0,1,0
4,246951,5,1,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246888,,1,
1,246889,,1,
2,246890,,,1
3,246933,,1,
4,246951,5,1,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 22 2022 10:59AM,247050,10,"Methapharm, Inc."
1,Sep 22 2022 10:58AM,247049,10,"Methapharm, Inc."
2,Sep 22 2022 10:56AM,247047,20,Alumier Labs Inc.
7,Sep 22 2022 10:56AM,247046,10,"Methapharm, Inc."
15,Sep 22 2022 10:55AM,247038,10,"Methapharm, Inc."
31,Sep 22 2022 10:54AM,247043,10,Eywa Pharma Inc.
33,Sep 22 2022 10:53AM,247044,18,Eye Pharma Inc
34,Sep 22 2022 10:46AM,247042,16,Sartorius Bioprocess Solutions
38,Sep 22 2022 10:40AM,247035,20,"HVL, LLC dba Atrium Innovations"
40,Sep 22 2022 10:39AM,247041,12,SugarBear


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 22 2022 10:59AM,247050,10,"Methapharm, Inc.",,,1
1,Sep 22 2022 10:58AM,247049,10,"Methapharm, Inc.",,,1
2,Sep 22 2022 10:56AM,247047,20,Alumier Labs Inc.,,,5
3,Sep 22 2022 10:56AM,247046,10,"Methapharm, Inc.",,,8
4,Sep 22 2022 10:55AM,247038,10,"Methapharm, Inc.",,,16
5,Sep 22 2022 10:54AM,247043,10,Eywa Pharma Inc.,,,2
6,Sep 22 2022 10:53AM,247044,18,Eye Pharma Inc,,,1
7,Sep 22 2022 10:46AM,247042,16,Sartorius Bioprocess Solutions,,,4
8,Sep 22 2022 10:40AM,247035,20,"HVL, LLC dba Atrium Innovations",,,2
9,Sep 22 2022 10:39AM,247041,12,SugarBear,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 10:59AM,247050,10,"Methapharm, Inc.",1,,
1,Sep 22 2022 10:58AM,247049,10,"Methapharm, Inc.",1,,
2,Sep 22 2022 10:56AM,247047,20,Alumier Labs Inc.,5,,
3,Sep 22 2022 10:56AM,247046,10,"Methapharm, Inc.",8,,
4,Sep 22 2022 10:55AM,247038,10,"Methapharm, Inc.",16,,
5,Sep 22 2022 10:54AM,247043,10,Eywa Pharma Inc.,2,,
6,Sep 22 2022 10:53AM,247044,18,Eye Pharma Inc,1,,
7,Sep 22 2022 10:46AM,247042,16,Sartorius Bioprocess Solutions,4,,
8,Sep 22 2022 10:40AM,247035,20,"HVL, LLC dba Atrium Innovations",2,,
9,Sep 22 2022 10:39AM,247041,12,SugarBear,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 10:59AM,247050,10,"Methapharm, Inc.",1,,
1,Sep 22 2022 10:58AM,247049,10,"Methapharm, Inc.",1,,
2,Sep 22 2022 10:56AM,247047,20,Alumier Labs Inc.,5,,
3,Sep 22 2022 10:56AM,247046,10,"Methapharm, Inc.",8,,
4,Sep 22 2022 10:55AM,247038,10,"Methapharm, Inc.",16,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 10:59AM,247050,10,"Methapharm, Inc.",1.0,NaN,NaN
1,Sep 22 2022 10:58AM,247049,10,"Methapharm, Inc.",1.0,NaN,NaN
2,Sep 22 2022 10:56AM,247047,20,Alumier Labs Inc.,5.0,NaN,NaN
3,Sep 22 2022 10:56AM,247046,10,"Methapharm, Inc.",8.0,NaN,NaN
4,Sep 22 2022 10:55AM,247038,10,"Methapharm, Inc.",16.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 22 2022 10:59AM,247050,10,"Methapharm, Inc.",1.0,0.0,0.0
1,Sep 22 2022 10:58AM,247049,10,"Methapharm, Inc.",1.0,0.0,0.0
2,Sep 22 2022 10:56AM,247047,20,Alumier Labs Inc.,5.0,0.0,0.0
3,Sep 22 2022 10:56AM,247046,10,"Methapharm, Inc.",8.0,0.0,0.0
4,Sep 22 2022 10:55AM,247038,10,"Methapharm, Inc.",16.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2964257,62.0,3.0,5.0
12,494081,2.0,0.0,0.0
15,494060,41.0,1.0,0.0
16,247042,4.0,0.0,0.0
18,247044,1.0,0.0,0.0
19,740925,7.0,2.0,0.0
20,741113,8.0,0.0,0.0
21,1481597,3.0,3.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2964257,62.0,3.0,5.0
1,12,494081,2.0,0.0,0.0
2,15,494060,41.0,1.0,0.0
3,16,247042,4.0,0.0,0.0
4,18,247044,1.0,0.0,0.0
5,19,740925,7.0,2.0,0.0
6,20,741113,8.0,0.0,0.0
7,21,1481597,3.0,3.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,62.0,3.0,5.0
1,12,2.0,0.0,0.0
2,15,41.0,1.0,0.0
3,16,4.0,0.0,0.0
4,18,1.0,0.0,0.0
5,19,7.0,2.0,0.0
6,20,8.0,0.0,0.0
7,21,3.0,3.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,62.0
1,12,Released,2.0
2,15,Released,41.0
3,16,Released,4.0
4,18,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,18,19,20,21
Status,,,,,,,,
Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,3.0,0.0,1.0,0.0,0.0,2.0,0.0,3.0
Released,62.0,2.0,41.0,4.0,1.0,7.0,8.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,18,19,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,0.0,1.0,0.0,0.0,2.0,0.0,3.0
2,Released,62.0,2.0,41.0,4.0,1.0,7.0,8.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,18,19,20,21
0,Completed,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,0.0,1.0,0.0,0.0,2.0,0.0,3.0
2,Released,62.0,2.0,41.0,4.0,1.0,7.0,8.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()